# Project Overview

Health care economists question whether the benefits of added medical care outweigh the costs. [Almond et al. (2008](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2903901/) propose a Regression Discontinuity Design (RDD) to compare health outcomes of newborns born around the threshold of 1500 grams. Those born under 1500 grams are considered very low birth weight and are consequently receive supplemental medical attention. Those born above the threshold are generally considered healthy. Notably, 1500 grams is a conventional measure not necessarily rooted in biology. 

This project provides an overview of RDD, evaluates its assumptions, re-produces findings from Almond et al., and presents thoughts on increased medical expenditures.  

# Regression Discontinuity Design Primer

## Background

RDD is common in political science and econometrics as a quasi-experimental design when randomization is not possible. The idea is to exploit a threshold in which units above and below the margin are assigned to "treatment" and "control" groups. Units that lie very close to the threshold are similar enough, on average, which allows researchers to identify the missing counterfactual required to estimate average treatment effects. As a result they may not vary much in terms of potential outcomes, and any differences in outcomes may be attributed to an intervention.

## Assumption

The key assumption for RDD to be valid is that the mean potential outcomes are continuous on the running variable. In other words, there must not be any discontinuous jump in our outcome variable when it is plotted against the variable that determines assignment to treatment or control plotted on the x axis. This assumption will be made clearer and evaluated later on.

# Loading and Checking Data

While [matplotlib](https://matplotlib.org) and [seaborn](https://seaborn.pydata.org) are popular plotting libraries for Python, I turn to [plotnine](https://plotnine.readthedocs.io/en/stable/#) for this project. My original analysis was in R, and ggplot created clear visualizations which I admired, and I wished to create them in Python. Plotnine is Python's version of R's ggplot, and the library utilizes a practically identical grammar of graphics. The library will need to be installed before running.

The original reserachers use data from the National Center for Health Statistics (NCHS) which contains data on 66 million births between 1983 a

In [1]:
import numpy as np
import os
import pandas as pd
# pip install plotnine
import plotnine as p9
import statsmodels.formula.api as smf

path = os.path.dirname(os.path.abspath("__file__"))

In [2]:
def birth_weight_loader():
    """Loads in stata file containg birth weight data."""

    data_path = os.path.join(path, 'almond_etal_2008.dta')
    df = pd.read_stata(data_path)

    return df 

In [3]:
df = birth_weight_loader() 
df

,yob,yod,staters,mom_age,mom_race,mom_ed,mom_ed1,mom_ed2,mom_ed3,mom_ed4,...,dad_age,dad_race,sex,plural,mom_origin,dad_origin,tot_order,live_order,pldel,attend
0,1983,NaN,1,21,black,12.0,0,1,0,0,...,20.0,black,2,1,88,88,2.0,2.0,Hospital Births,Physician
1,1983,NaN,1,34,white,12.0,0,1,0,0,...,37.0,white,2,1,88,88,4.0,3.0,Hospital Births,Physician
2,1983,NaN,10,31,white,12.0,0,1,0,0,...,25.0,white,2,1,88,88,4.0,3.0,Hospital Births,Physician
3,1983,NaN,1,18,black,11.0,1,0,0,0,...,NaN,other,2,1,88,88,1.0,1.0,Hospital Births,Physician
4,1983,NaN,1,17,black,9.0,1,0,0,0,...,NaN,other,2,1,88,88,2.0,2.0,Hospital Births,Physician
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376403,2002,2002.0,50,33,white,17.0,0,0,0,1,...,32.0,white,1,1,0,0,1.0,1.0,Hospital Births,Physician
376404,2002,2002.0,50,19,black,10.0,1,0,0,0,...,NaN,other,1,1,0,9,2.0,2.0,Hospital Births,Physician
376405,2002,2003.0,50,34,white,12.0,0,1,0,0,...,NaN,white,1,1,0,0,4.0,4.0,Hospital Births,Physician
376406,2002,2003.0,50,26,white,13.0,0,0,1,0,...,NaN,white,1,2,0,0,1.0,1.0,Hospital Births,Physician


Let's start by getting an idea of the distribution of birth weights in the sample.

In [4]:
def get_descriptive_stats(dataframe, column):
    """Returns descriptive statistics for a specified column in dataframe."""

    df = dataframe.copy()
    descriptive_stats = df[column].describe()

    return descriptive_stats

In [6]:
get_descriptive_stats(df, 'yob')

count    376408.000000
mean       1992.986289
std           6.181239
min        1983.000000
25%        1987.000000
50%        1995.000000
75%        1999.000000
max        2002.000000
Name: yob, dtype: float64